In [1]:
"""
Created on August 4th 2022

@author: JJ
"""

'\nCreated on August 4th 2022\n\n@author: JJ\n'

In [2]:
import os, re, sys
import numpy as np
import pandas as pd
from math import log
import random
import scipy.io
import tensorflow as tf
import csv

2022-08-30 14:45:01.990722: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.regularizers import (
    l2, 
    l1, 
    l1_l2
)
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import (
    activations, 
    initializers, 
    regularizers, 
    constraints
)

In [3]:
#import from python files
from model import build_model

In [4]:
num_threads = 4
# Maximum number of threads to use for OpenMP parallel regions.
os.environ["OMP_NUM_THREADS"] = "4"
# Without setting below 2 environment variables, it didn't work for me. Thanks to @cjw85 
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"
os.environ["TF_NUM_INTEROP_THREADS"] = "4"

tf.config.threading.set_inter_op_parallelism_threads(
    num_threads
)
tf.config.threading.set_intra_op_parallelism_threads(
    num_threads
)
tf.config.set_soft_device_placement(True)

In [5]:
# in a well-defined initial state.
random.seed
np.random.seed(1337)
tf.random.set_seed(1337)

In [6]:
import matplotlib
import matplotlib.pyplot as plt

In [7]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping
)
from sklearn.metrics import (
    roc_curve,
    auc,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
)

In [8]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample, shuffle
from sklearn.feature_selection import (
    SelectKBest,
    chi2
)
import keras_tuner as kt
from collections import defaultdict

In [ ]:
#import pickle
#with open ('fulldata_pseudoseq_dictionary.pkl', 'rb') as f:
#    pseudo_seq = pickle.load(f)

In [ ]:
import pickle
with open ('fulldata_pseudoseq_dictionary_converted.pkl', 'rb') as f:
    pseudo_seq = pickle.load(f)

In [ ]:
type(pseudo_seq)

In [ ]:
    training_data = []
    for allele in pseudo_seq.keys():
        allele_data = pseudo_seq[allele]
        random.shuffle(allele_data)
        allele_data = np.array(allele_data)
        training_data.extend(allele_data)
    
    [all_pep, all_mhc, all_target] = [[i[j] for i in training_data] for j in range(3)]
    all_pep = np.array(all_pep)
    all_mhc = np.array(all_mhc)
    all_target = np.array(all_target)

In [ ]:
print(all_pep.shape)
print(all_mhc.shape)
print(all_target.shape)

In [ ]:
#np.save('/home/jjia1/jjia1/viralepitope/all_pep.npy', all_pep)
#np.save('/home/jjia1/jjia1/viralepitope/all_mhc.npy', all_mhc)
#np.save('/home/jjia1/jjia1/viralepitope/all_target.npy', all_target)

In [ ]:
np.save('/home/jjia1/jjia1/viralepitope/all_pep_converted.npy', all_pep)
np.save('/home/jjia1/jjia1/viralepitope/all_mhc_converted.npy', all_mhc)
np.save('/home/jjia1/jjia1/viralepitope/all_target_converted.npy', all_target)

In [ ]:
#all_pep, all_mhc, all_target = np.load('/home/jjia1/jjia1/viralepitope/all_pep.npy'), np.load('/home/jjia1/jjia1/viralepitope/all_mhc.npy'), np.load('/home/jjia1/jjia1/viralepitope/all_target.npy')

In [9]:
all_pep, all_mhc, all_target = np.load('/home/jjia1/jjia1/viralepitope/all_pep_converted.npy'), np.load('/home/jjia1/jjia1/viralepitope/all_mhc_converted.npy'), np.load('/home/jjia1/jjia1/viralepitope/all_target_converted.npy')

In [ ]:
#build 2 models
#one model for the sequence, one model for the MHC names
#integrate the outputs in a final dense layer

In [10]:
def create_model(training_pep, training_mhc):
        filters, kernel_size, fc1_size, fc2_size, fc3_size = 128, 3, 256, 64, 2
        inputs_1 = Input(shape = (np.shape(training_pep[0])[0],20))
        inputs_2 = Input(shape = (34,np.shape(training_mhc[0])[0]))
        #Initial feature extraction using a convolutional layer
        pep_conv = Conv1D(filters,kernel_size,padding = 'same',activation = 'relu',strides = 1)(inputs_1)
        pep_maxpool = MaxPooling1D()(pep_conv)
        mhc_conv_1 = Conv1D(filters,kernel_size,padding = 'same',activation = 'relu',strides = 1)(inputs_2)
        mhc_maxpool_1 = MaxPooling1D()(mhc_conv_1)
        #The convolutional module
        mhc_conv_2 = Conv1D(filters,kernel_size,padding = 'same',activation = 'relu',strides = 1)(mhc_maxpool_1)
        mhc_maxpool_2 = MaxPooling1D()(mhc_conv_2)
        flat_pep_0 = Flatten()(pep_conv)
        flat_pep_1 = Flatten()(pep_conv)
        flat_pep_2 = Flatten()(pep_conv)
        flat_mhc_0 = Flatten()(inputs_2)
        flat_mhc_1 = Flatten()(mhc_maxpool_1)
        flat_mhc_2 = Flatten()(mhc_maxpool_2)
        cat_0 = Concatenate()([flat_pep_0, flat_mhc_0])
        cat_1 = Concatenate()([flat_pep_1, flat_mhc_1])
        cat_2 = Concatenate()([flat_pep_2, flat_mhc_2])        
        fc1_0 = Dense(fc1_size,activation = "relu")(cat_0)
        fc1_1 = Dense(fc1_size,activation = "relu")(cat_1)
        fc1_2 = Dense(fc1_size,activation = "relu")(cat_2)
        merge_1 = Concatenate()([fc1_0, fc1_1, fc1_2])
        fc2 = Dense(fc2_size,activation = "relu")(merge_1)
        fc3 = Dense(fc3_size,activation = "relu")(fc2)
        #The attention module
        mhc_attention_weights = Flatten()(TimeDistributed(Dense(1))(mhc_conv_1))
        pep_attention_weights = Flatten()(TimeDistributed(Dense(1))(pep_conv))
        mhc_attention_weights = Activation('softmax')(mhc_attention_weights)
        pep_attention_weights = Activation('softmax')(pep_attention_weights)        
        mhc_conv_permute = Permute((2,1))(mhc_conv_1)
        pep_conv_permute = Permute((2,1))(pep_conv)
        mhc_attention = Dot(-1)([mhc_conv_permute, mhc_attention_weights])
        pep_attention = Dot(-1)([pep_conv_permute, pep_attention_weights])
        #Concatenating the output of the two modules
        merge_2 = Concatenate()([mhc_attention,pep_attention,fc3])
        #Output of the model
        out = Dense(1,activation = "sigmoid")(merge_2)
        model = Model(inputs=[inputs_1, inputs_2],outputs=out)  
        model.summary()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])

In [11]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 5)

In [12]:
train_pep, test_pep, train_mhc, test_mhc, train_target, test_target = train_test_split(all_pep, all_mhc, all_target, test_size = 0.2, stratify= all_target)

In [13]:
print(train_pep.shape)
print(test_pep.shape)

(756347, 30, 20)
(189087, 30, 20)


In [14]:
folder = "model/CV"

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1337)    
allprobas_=np.array([]) 
all_labels=np.array([])

In [19]:
for i, (train, test) in enumerate(kfold.split(train_pep, train_target)):
    training_pep = train_pep[train]
    training_mhc = train_mhc[train]
    training_target = train_target[train]
    
    validation_pep = train_pep[test]
    validation_mhc = train_mhc[test]
    validation_target = train_target[test]

    mc = ModelCheckpoint(folder + '/model_' +str(i)+'.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    model = build_model(training_pep, training_mhc)
    model.summary()

    model.fit([training_pep,training_mhc], 
            training_target,
            batch_size=512,
            epochs = 100,
            shuffle=True,
            callbacks=[es, mc],
            validation_data=([validation_pep,validation_mhc], validation_target),
            verbose=1)

    saved_model = load_model(folder + '/model_' +str(i)+'.h5')
    probas_ = saved_model.predict([np.array(validation_pep),np.array(validation_mhc)])
    allprobas_ = np.append(allprobas_, probas_)           
    allylable = np.append(allylable, validation_target)
    del model


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
peptide (InputLayer)            [(None, 30, 20)]     0                                            
__________________________________________________________________________________________________
mhc (InputLayer)                [(None, 34, 20)]     0                                            
__________________________________________________________________________________________________
conv1_pep (Conv1D)              (None, 30, 128)      7808        peptide[0][0]                    
__________________________________________________________________________________________________
conv1_mhc (Conv1D)              (None, 34, 128)      7808        mhc[0][0]                        
____________________________________________________________________________________________

2022-08-30 15:30:44.755345: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1452184800 exceeds 10% of free system memory.


Epoch 1/100
1182/1182 [==============================] - 113s 94ms/step - loss: 0.2693 - accuracy: 0.9092 - auc: 0.7588 - val_loss: 0.2128 - val_accuracy: 0.9140 - val_auc: 0.8826

Epoch 00001: val_accuracy improved from -inf to 0.91398, saving model to model/CV/model_0.h5
Epoch 2/100
1182/1182 [==============================] - 107s 91ms/step - loss: 0.2167 - accuracy: 0.9142 - auc: 0.8734 - val_loss: 0.2055 - val_accuracy: 0.9153 - val_auc: 0.8931

Epoch 00002: val_accuracy improved from 0.91398 to 0.91532, saving model to model/CV/model_0.h5
Epoch 3/100
1182/1182 [==============================] - 104s 88ms/step - loss: 0.2117 - accuracy: 0.9146 - auc: 0.8823 - val_loss: 0.2022 - val_accuracy: 0.9166 - val_auc: 0.8968

Epoch 00003: val_accuracy improved from 0.91532 to 0.91661, saving model to model/CV/model_0.h5
Epoch 4/100
1182/1182 [==============================] - 101s 85ms/step - loss: 0.2092 - accuracy: 0.9156 - auc: 0.8870 - val_loss: 0.2013 - val_accuracy: 0.9159 - val_auc:

In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

########ROC_figure
figure1, ax1 = plt.subplots(figsize=figsize)
ax1.tick_params(labelsize=18)
labels = ax1.get_xticklabels() + ax1.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]  

fpr, tpr, thresholds = roc_curve(allylable, allprobas_)
roc_auc = auc(fpr, tpr)
print(roc_auc)

ax1.plot(fpr, tpr, color='b',
    label=r'Mean ROC (AUC = %0.4f)' % (roc_auc),
    lw=2, alpha=.8)
ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)
ax1.set_xlim([-0.05, 1.05])
ax1.set_ylim([-0.05, 1.05])
ax1.set_xlabel('False Positive Rate', font1)
ax1.set_ylabel('True Positive Rate', font1)
title1 = 'Cross Validated ROC Curve'
ax1.set_title(title1, font1)
ax1.legend(loc="lower right")
figure1.savefig(folder + '5_fold_roc.png', dpi=300, bbox_inches = 'tight')

########PR_figure
figure2, ax2 = plt.subplots(figsize=figsize)
ax2.tick_params(labelsize=18)
labels = ax2.get_xticklabels() + ax2.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels] 

precision, recall, _ = precision_recall_curve(allylable, allprobas_)
ax2.plot(recall, precision, color='b',
        label=r'Precision-Recall (AUC = %0.4f)' % (average_precision_score(allylable, allprobas_)),
        lw=2, alpha=.8)

ax2.set_xlim([-0.05, 1.05])
ax2.set_ylim([-0.05, 1.05])
ax2.set_xlabel('Recall', font1)
ax2.set_ylabel('Precision', font1)
title2 = 'Cross Validated PR Curve'
ax2.set_title(title2, font1)
ax2.legend(loc="lower left")
figure2.savefig(folder + '5_fold_pr.png', dpi=300, bbox_inches = 'tight')


In [ ]:
font1 = {'family' : 'Times New Roman',
        'weight' : 'normal',
        'size'   : 16}
figsize=6.2, 6.2

########ROC_figure
figure1, ax1 = plt.subplots(figsize=figsize)
ax1.tick_params(labelsize=18)
labels = ax1.get_xticklabels() + ax1.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels]  

fpr, tpr, thresholds = roc_curve(all_labels, allprobas_)
roc_auc = auc(fpr, tpr)
print(roc_auc)

ax1.plot(fpr, tpr, color='b',
    label=r'ROC (AUC = %0.4f)' % (roc_auc),
    lw=2, alpha=.8)
ax1.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Luck', alpha=.8)
ax1.set_xlim([-0.05, 1.05])
ax1.set_ylim([-0.05, 1.05])
ax1.set_xlabel('False Positive Rate', font1)
ax1.set_ylabel('True Positive Rate', font1)
# title1 = 'Cross Validated ROC Curve'
# ax1.set_title(title1, font1)
ax1.legend(loc="lower right")

In [ ]:
########PR_figure
figure2, ax2 = plt.subplots(figsize=figsize)
ax2.tick_params(labelsize=18)
labels = ax2.get_xticklabels() + ax2.get_yticklabels()
[label.set_fontname('Times New Roman') for label in labels] 

precision, recall, _ = precision_recall_curve(all_labels, allprobas_)
ax2.plot(recall, precision, color='b',
        label=r'Precision-Recall (AUC = %0.4f)' % (average_precision_score(all_labels, allprobas_)),
        lw=2, alpha=.8)

ax2.set_xlim([-0.05, 1.05])
ax2.set_ylim([-0.05, 1.05])
ax2.set_xlabel('Recall', font1)
ax2.set_ylabel('Precision', font1)
# title2 = 'Cross Validated PR Curve'
# ax2.set_title(title2, font1)
ax2.legend(loc="lower left")

In [ ]:
print(saved_model.history.keys())

In [ ]:
plt.plot(saved_model.history["loss"])
plt.plot(saved_model.history["val_loss"])
plt.legend(['train', 'test'], loc='best')

In [ ]:
plt.plot(saved_model.history["accuracy"])
plt.plot(saved_model.history["val_accuracy"])
plt.legend(['train', 'test'], loc='best')